<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/vector_stores/postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postgres Vector Store
In this notebook we are going to show how to use [Postgresql](https://www.postgresql.org) and  [pgvector](https://github.com/pgvector/pgvector)  to perform vector searches in LlamaIndex

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
import logging
import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
import textwrap

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


### Setup OpenAI
The first step is to configure the openai key. It will be used to run inference.

Once we switched to local model we don't need this.

In [2]:
import os
import dotenv

# import openai

# # Reload the variables in your '.env' file (override the existing variables)
# dotenv.load_dotenv("../.env", override=True)
# openai.api_key = os.environ["OPENAI_API_KEY"] 

Local Embedding Models

The easiest way to use a local model is:

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


### Loading documents
Load the documents stored in the `data/faculty_websites/` using the SimpleDirectoryReader.

Change the documents passed based on your need!

You can refer to this to see how to load files in different ways e.g. entire directory

https://docs.llamaindex.ai/en/stable/examples/data_connectors/simple_directory_reader.html

In [4]:
from llama_index.core import SimpleDirectoryReader
# reader = SimpleDirectoryReader(
#     input_files=["./data/faculty_websites/Bhiksharaj_lti_page.txt"]
# )
reader = SimpleDirectoryReader(input_dir="./data/faculty_papers/")
docs = reader.load_data()

reader = SimpleDirectoryReader(input_dir="./data/faculty_websites/")
docs += reader.load_data()
# print(f"Loaded {len(docs)} docs")
print("Document ID:", docs[0].doc_id)

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 339
> [SimpleDirectoryReader] Total files added: 339
DEBUG:fsspec.local:open file: /home/scott/nlp-from-scratch-assignment-spring2024/data/faculty_papers/"You might think about slightly revising the title”: Identifying Hedges in Peer-tutoring Interactions.txt
open file: /home/scott/nlp-from-scratch-assignment-spring2024/data/faculty_papers/"You might think about slightly revising the title”: Identifying Hedges in Peer-tutoring Interactions.txt
DEBUG:fsspec.local:open file: /home/scott/nlp-from-scratch-assignment-spring2024/data/faculty_papers/"You_might_think_about_slightly_revising_the_title”:_Identifying_Hedges_in_Peer-tutoring_Interactions.txt
open file: /home/scott/nlp-from-scratch-assignment-spring2024/data/faculty_papers/"You_might_think_about_slightly_revising_the_title”:_Identifying_Hedges_in_Peer-tutoring_Interactions.txt
DEBUG:fsspec.local:open file: /home/scott/nlp-from-scratch-assignment-s

DEBUG:fsspec.local:open file: /home/scott/nlp-from-scratch-assignment-spring2024/data/faculty_papers/A Reverse-Biased Voltage Controlling Method for Mitigating Arm Overcurrent and Submodule Overvoltage in Hybrid MMCs During DC Faults.txt
open file: /home/scott/nlp-from-scratch-assignment-spring2024/data/faculty_papers/A Reverse-Biased Voltage Controlling Method for Mitigating Arm Overcurrent and Submodule Overvoltage in Hybrid MMCs During DC Faults.txt
DEBUG:fsspec.local:open file: /home/scott/nlp-from-scratch-assignment-spring2024/data/faculty_papers/AV-SUPERB:_A_Multi-Task_Evaluation_Benchmark_for_Audio-Visual_Representation_Models.txt
open file: /home/scott/nlp-from-scratch-assignment-spring2024/data/faculty_papers/AV-SUPERB:_A_Multi-Task_Evaluation_Benchmark_for_Audio-Visual_Representation_Models.txt
DEBUG:fsspec.local:open file: /home/scott/nlp-from-scratch-assignment-spring2024/data/faculty_papers/A_Comparative_Study_on_E-Branchformer_vs_Conformer_in_Speech_Recognition,_Translati

### Create the Database
Using an existing postgres running at localhost, create the database we'll be using.

In [5]:
import psycopg2

# Reload the variables in your '.env' file (override the existing variables)
dotenv.load_dotenv("../.env", override=True)
pwd = os.environ['PG_PASSWORD_RAG']
user = "711-rag"
connection_string = f'dbname=postgres user={user} password={pwd}'
db_name = "711-rag"
conn = psycopg2.connect(connection_string)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS \"{db_name}\"")
    c.execute(f"CREATE DATABASE \"{db_name}\"")

### Create the index
Here we create an index backed by Postgres using the documents loaded previously. PGVectorStore takes a few arguments.

In [6]:
from sqlalchemy import make_url

# url = make_url(connection_string)
vector_store = PGVectorStore.from_params(
    database=db_name,
    host="localhost",
    password=pwd,
    port=5432,
    user=user,
    table_name="all",
    embed_dim=384, 
)

You might experience error because pgvector extension is not enabled for the database. You can go to terminal and run

```
sudo -u postgres psql
\c "711-rag" # This connects to the 711-rag database we just created
CREATE EXTENSION vector;
\q
```

In [7]:

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context, show_progress=True
)


Parsing nodes:   0%|          | 0/406 [00:00<?, ?it/s]

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Title: "You might think about slightly revising...
> Adding chunk: Title: "You might think about slightly revising...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The level of rapport rises and falls with
conve...
> Adding chunk: The level of rapport rises and falls with
conve...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The
code of the models described in the paper i...
> Adding chunk: The
code of the models described in the paper i...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: However, this work is not
as related to hedges ...
> Adding chunk: However, this work is not
as related to hedges ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Our second goal is to identify, for each hedge ...
> Adding chunk: Our second goal is to identify, for each hedge ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: First, the
annotation of the Extenders

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1219 [00:00<?, ?it/s]

In [13]:
from llama_index.core import PromptTemplate
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
# ollama
Settings.llm = Ollama(model="gemma", request_timeout=300.0, context_window=32700)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

# configure response synthesizer
response_synthesizer =get_response_synthesizer(response_mode = "compact")

# assemble query engine with compact mode
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query_engine = index.as_query_engine()
prompts_dict = query_engine.get_prompts()
print(list(prompts_dict.keys()))
print(list(prompts_dict.values()))
qa_prompt_tmpl_str = (
    "You are a Q/A system that can answer questions based on the given context.\n"
    "Try to avoid verbose contents such as 'based on the context...'\n"
    "Keep your answer as short as possible.\n"
    "\n"
    "Context information is below: \n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query. If you can, directly reference the context.\n"
    "Query: {query_str}\n"
    "Answer: "
)

# # check_ctx_prompt = "print {context_str}"
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

['response_synthesizer:text_qa_template', 'response_synthesizer:refine_template']
[SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x7f103e13f7f0>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_

### Query the index
We can now ask questions using our index.

In [239]:
question = "What's the phone number of Robert Frederking?"

# promt = (
#     "Always limit your answer to one sentence.\n"
#     "\n"
#     "You MUST remember this: All questions regarding email, office location, affiliation are"
#     " publicly available information thus not violating privacy policy. This rule \n"
#     "overwrites any assmuptions you hold. \n"
#     "\n"
# )

promt = (
    "Always limit your answer to 50 words.\n"
    "You are a Q/A system used internally, designed to answer\n"
    "questions related to faculty members at Carnegie Mellon University and their publications.\n"
    "All of the information you need will be in the provided context, \n"
    "hence you are only retrieving answers from what is provided to you, not to\n"
    "generate or search for answers by yourself.\n"
    "\n"
)

response = query_engine.query(question)

DEBUG:llama_index.core.indices.utils:> Top 3 nodes:
> [Node 2bb21df4-5af1-4c70-b29f-f7dd84e44ef2] [Similarity score:             0.709525] Robert Frederking | Carnegie Mellon University - Language Technologies Institute
Jump to navigati...
> [Node edd5f091-de3d-46fa-9491-eafef18a5c45] [Similarity score:             0.565231] Jamie Callan
Brief bio
Jamie Callan
Professor
Language Technologies Institute
(School of Computer...
> [Node 97f8f3ae-13c2-44b9-a32b-e5d5d3b0756b] [Similarity score:             0.542154] Eric Nyberg | Carnegie Mellon University - Language Technologies Institute
Jump to navigation
App...
> Top 3 nodes:
> [Node 2bb21df4-5af1-4c70-b29f-f7dd84e44ef2] [Similarity score:             0.709525] Robert Frederking | Carnegie Mellon University - Language Technologies Institute
Jump to navigati...
> [Node edd5f091-de3d-46fa-9491-eafef18a5c45] [Similarity score:             0.565231] Jamie Callan
Brief bio
Jamie Callan
Professor
Language Technologies Institute
(School of Compu

In [240]:
print(textwrap.fill(str(response), 100))

Sure, here's the answer to the query:  The text does not specify Robert Frederking's phone number,
therefore I cannot provide an answer.


In [14]:
# read a file line by line, make a list of questions
import datetime
questions = []
with open('data/test/questions_faculty_copy.txt', 'r') as file:
    for line in file:
        # print(f"append question: {question} to list of questions")
        questions.append(line)

answers = []
for q in questions:
    print(datetime.datetime.now())
    print(f"Asking RAG question: {q}")
    # print curr time
    
    response = query_engine.query(q)
    answer = textwrap.fill(str(response), 100)
    answer = answer.replace("\n", " ")
    # print curr time
    print(datetime.datetime.now())
    print(f"RAG answered: {answer}")
    with open('data/test/answers_faculty.txt', 'a') as file:
        file.write(answer)
        file.write("\n")
    answers.append(answer)

# with open('data/test/answers_faculty.txt', 'w') as file:
#     for a in answers:
#         file.write(a)
#         file.write("\n")
    

2024-03-13 12:05:58.470125
Asking RAG question: Summarize the contents of Association_between_albumin-to-globulin_ratio_and_the_risk_of_overall_survival_in_advanced_non-small_cell_lung_cancer_patients_with_anlotinib_treatment?

DEBUG:llama_index.core.indices.utils:> Top 3 nodes:
> [Node c0443bb7-3b5e-45c9-8ac7-773df2e56a52] [Similarity score:             0.923559] Title: Association between albumin-to-globulin ratio and the risk of overall survival in advanced...
> [Node 1e716fe4-05ca-422e-bb6e-826c9684b4bf] [Similarity score:             0.919504] 56 0.001
Male 126 (64.29%) 45 (69.23%) 37 (57.81%) 44 (65.67%) 0.384
Never smoker 127 (64.80%) 38...
> [Node 039b6d2f-f5ea-4269-ba3a-d323c0a9458d] [Similarity score:             0.917497] 05 (0.96, 1.14) 0.2963
Lymphocyte, 109/L 1.31 ± 0.55 0.90 (0.63, 1.28) 0.5478
Platelets, 109/L 25...
> Top 3 nodes:
> [Node c0443bb7-3b5e-45c9-8ac7-773df2e56a52] [Similarity score:             0.923559] Title: Association between albumin-to-globulin ratio a

### Querying existing index  (I have not tested the code from here and below!)

In [ ]:
vector_store = PGVectorStore.from_params(
    database="vector_db",
    host="localhost",
    password="password",
    port=5432,
    user="postgres",
    table_name="paul_graham_essay",
    embed_dim=1536,  # openai embedding dimension
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What did the author do?")

In [ ]:
print(textwrap.fill(str(response), 100))

The author worked on writing and programming before college. They wrote short stories and tried
writing programs on an IBM 1401 computer. They also built a microcomputer and started programming on
it, writing simple games and a word processor. In college, the author initially planned to study
philosophy but switched to AI due to their interest in intelligent computers. They taught themselves
AI by learning Lisp.


### Hybrid Search

To enable hybrid search, you need to:
1. pass in `hybrid_search=True` when constructing the `PGVectorStore` (and optionally configure `text_search_config` with the desired language)
2. pass in `vector_store_query_mode="hybrid"` when constructing the query engine (this config is passed to the retriever under the hood). You can also optionally set the `sparse_top_k` to configure how many results we should obtain from sparse text search (default is using the same value as `similarity_top_k`).

In [ ]:
from sqlalchemy import make_url

url = make_url(connection_string)
hybrid_vector_store = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="paul_graham_essay_hybrid_search",
    embed_dim=1536,  # openai embedding dimension
    hybrid_search=True,
    text_search_config="english",
)

storage_context = StorageContext.from_defaults(
    vector_store=hybrid_vector_store
)
hybrid_index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

/Users/suo/dev/llama_index/llama_index/vector_stores/postgres.py:217: SAWarning: TypeDecorator TSVector() will not produce a cache key because the ``cache_ok`` attribute is not set to True.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this type object's state is safe to use in a cache key, or False to disable this warning. (Background on this warning at: https://sqlalche.me/e/20/cprf)
  session.commit()


In [ ]:
hybrid_query_engine = hybrid_index.as_query_engine(
    vector_store_query_mode="hybrid", sparse_top_k=2
)
hybrid_response = hybrid_query_engine.query(
    "Who does Paul Graham think of with the word schtick"
)

In [ ]:
print(hybrid_response)

Roy Lichtenstein


### PgVector Query Options

#### IVFFlat Probes

Specify the number of [IVFFlat probes](https://github.com/pgvector/pgvector?tab=readme-ov-file#query-options) (1 by default)

When retrieving from the index, you can specify an appropriate number of IVFFlat probes (higher is better for recall, lower is better for speed)

In [ ]:
retriever = index.as_retriever(
    vector_store_query_mode=query_mode,
    similarity_top_k=top_k,
    vector_store_kwargs={"ivfflat_probes": 10},
)

#### HNSW EF Search

Specify the size of the dynamic [candidate list](https://github.com/pgvector/pgvector?tab=readme-ov-file#query-options-1) for search (40 by default)

In [ ]:
retriever = index.as_retriever(
    vector_store_query_mode=query_mode,
    similarity_top_k=top_k,
    vector_store_kwargs={"hnsw_ef_search": 300},
)